# Import / Config

In [5]:
import datetime
import pandas as pd
from neo4j import GraphDatabase
import cred_neo4j as c

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## Test

In [ ]:
q = """
MATCH (p:Person {id: $personId}), (friend:Person {firstName: $firstName})
RETURN p.lastName, friend.lastName
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, personId=14, firstName="Jose")
        df = pd.DataFrame(result.data())
df

## Interactive / complex / 1

In [4]:
q = """
MATCH (p:Person {id: $personId}), (friend:Person {firstName: $firstName})
       WHERE NOT p=friend
       WITH p, friend
       MATCH path = shortestPath((p)-[:KNOWS*1..3]-(friend))
       WITH min(length(path)) AS distance, friend
ORDER BY
    distance ASC,
    friend.lastName ASC,
    toInteger(friend.id) ASC
LIMIT 20

MATCH (friend)-[:IS_LOCATED_IN]->(friendCity:City)
OPTIONAL MATCH (friend)-[studyAt:STUDY_AT]->(uni:University)-[:IS_LOCATED_IN]->(uniCity:City)
WITH friend, collect(
    CASE
        WHEN uni IS NULL THEN null
        ELSE [uni.name, studyAt.classYear, uniCity.name]
    END ) AS unis, friendCity, distance

OPTIONAL MATCH (friend)-[workAt:WORK_AT]->(company:Company)-[:IS_LOCATED_IN]->(companyCountry:Country)
WITH friend, collect(
    CASE
        WHEN company IS NULL then null
        ELSE [company.name, workAt.workFrom, companyCountry.name]
    END ) AS companies, unis, friendCity, distance

RETURN
    friend.id AS friendId,
    friend.lastName AS friendLastName,
    distance AS distanceFromPerson,
    friend.birthday AS friendBirthday,
    friend.creationDate AS friendCreationDate,
    friend.gender AS friendGender,
    friend.browserUsed AS friendBrowserUsed,
    friend.locationIP AS friendLocationIp,
    friend.email AS friendEmails,
    friend.speaks AS friendLanguages,
    friendCity.name AS friendCityName,
    unis AS friendUniversities,
    companies AS friendCompanies
ORDER BY
    distanceFromPerson ASC,
    friendLastName ASC,
    toInteger(friendId) ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, personId=14, firstName="Jose")
        df = pd.DataFrame(result.data())
df

,friendId,friendLastName,distanceFromPerson,friendBirthday,friendCreationDate,friendGender,friendBrowserUsed,friendLocationIp,friendEmails,friendLanguages,friendCityName,friendUniversities,friendCompanies
0,30786325577740,Alonso,2,1987-08-18,2012-06-26T14:37:35.179000000+02:00,female,Internet Explorer,196.1.135.241,Jose30786325577740@hotmail.com;Jose30786325577...,es;en,Jagüey_Grande,"[[University_of_Cienfuegos, None, Cienfuegos]]","[[Cubana_de_Aviación, None, Cuba], [Aerogaviot..."
1,4398046513564,Alves,2,1988-06-21,2010-07-03T06:25:52.859000000+02:00,male,Firefox,143.54.229.19,Jose4398046513564@gmail.com;Jose4398046513564@...,pt;en,Rio_de_Janeiro,[],"[[Flex_Linhas_Aéreas, None, Brazil], [Rio_Sul_..."
2,1380,Carvalho,2,1981-05-29,2010-02-20T15:01:08.026000000+01:00,male,Firefox,94.126.172.0,Jose1380@gmail.com,pt;en,Mesa,"[[Sabena_Flight_Academy, None, Mesa]]","[[Luzair, None, Portugal], [Hi_Fly_(airline), ..."
3,2199023260111,Carvalho,2,1987-05-22,2010-03-31T09:35:28.250000000+02:00,male,Firefox,88.214.132.137,Jose2199023260111@gmail.com;Jose2199023260111@...,pt;en,Mesa,"[[University_of_the_Algarve, None, Faro]]","[[NetJets_Europe, None, Portugal]]"
4,19791209301504,Costa,2,1984-07-19,2011-08-25T10:15:56.574000000+02:00,male,Opera,198.50.19.91,Jose19791209301504@priest.com;Jose197912093015...,pt;en,Campinas,"[[University_of_the_State_of_Amazonas, None, M...","[[Rico_Linhas_Aéreas, None, Brazil], [SAVAG_–_..."
5,10995116284575,Dom Pedro II,2,1980-02-28,2011-01-03T12:52:19.738000000+01:00,male,Chrome,200.0.68.13,Jose10995116284575@yahoo.com,pt;en,Passos,[],"[[TEAM_Linhas_Aéreas, None, Brazil]]"
6,19791209308329,Dom Pedro II,2,1989-07-25,2011-08-21T20:49:52.400000000+02:00,male,Firefox,177.6.108.225,Jose19791209308329@gmail.com;Jose1979120930832...,pt;en,Manaus,[[Pontifical_Catholic_University_of_Rio_de_Jan...,"[[Whitejets, None, Brazil], [Avianca_Brazil, N..."
7,24189255818493,Fernandez,2,1987-12-15,2011-11-21T22:34:06.857000000+01:00,female,Chrome,200.55.175.244,Jose24189255818493@gmail.com;Jose2418925581849...,es;en,Pinar_del_Río,"[[University_of_Ciego_de_Ávila, None, Ciego_de...","[[Aero_Caribbean, None, Cuba], [Cubana_de_Avia..."
8,2199023260364,Garcia,2,1988-04-19,2010-05-01T18:06:45.403000000+02:00,female,Firefox,111.68.47.44,Jose2199023260364@gmail.com;Jose2199023260364@...,en;tl,Cagayan_de_Oro,"[[Marcelo_H._del_Pilar_National_High_School, N...","[[Airphil_Express, None, Philippines], [Air_Ma..."
9,17592186045969,Garcia,2,1989-06-14,2011-05-31T14:24:34.978000000+02:00,male,Internet Explorer,148.230.139.205,Jose17592186045969@gmail.com;Jose1759218604596...,es;en,San_Pedro_Garza_García,"[[Technological_Institute_of_Mexicali, None, M...","[[MasAir, None, Mexico], [Aeromar, None, Mexic..."


## Interactive / complex / 2

In [13]:
d = datetime.datetime.strpdatetime("2010-10-16", "%Y-%m-%d")
d

AttributeError: type object 'datetime.datetime' has no attribute 'strpdatetime'

In [14]:
d = datetime.datetime.strptime("2020-10-16", "%Y-%m-%d")
q = """
MATCH (m:Message)
where m.creationDate < datetime($maxDate)
RETURN m
LIMIT 1
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, maxDate="2020-10-16")
        df = pd.DataFrame(result.data())
df

,m
0,"{'length': 140, 'browserUsed': 'Firefox', 'lan..."


In [16]:
q = """
/*
:params { personId: 10995116278009, maxDate: 1287230400000 }
*/
MATCH (:Person {id: $personId })-[:KNOWS]-(friend:Person)<-[:HAS_CREATOR]-(message:Message)
    WHERE message.creationDate < datetime($maxDate)
    RETURN
        friend.id AS personId,
        friend.firstName AS personFirstName,
        friend.lastName AS personLastName,
        message.id AS postOrCommentId,
        coalesce(message.content,message.imageFile) AS postOrCommentContent,
        message.creationDate AS postOrCommentCreationDate
    ORDER BY
        postOrCommentCreationDate DESC,
        toInteger(postOrCommentId) ASC
    LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, personId=14, maxDate="2010-10-16")
        df = pd.DataFrame(result.data())
df

,personId,personFirstName,personLastName,postOrCommentId,postOrCommentContent,postOrCommentCreationDate
0,8796093030850,Monja,Tsiranana,549758485894,"About Rumi, d in the year Rumi was born, his f...",2010-10-15T07:25:54.607000000+02:00
1,8796093030850,Monja,Tsiranana,549758485736,"About Rumi, Dervishes, famous for its Sufi da...",2010-10-15T02:09:18.473000000+02:00
2,8796093030850,Monja,Tsiranana,549758483700,"About Rumi, nd cultural language in the Persia...",2010-10-13T16:29:57.457000000+02:00
3,4398046517420,Irina,Markov,549756749671,"About Jorge Luis Borges, ry. In fact, critic A...",2010-10-13T10:23:43.715000000+02:00
4,2199023264850,Gheorghe,Popescu,549756812145,duh,2010-10-12T18:37:39.740000000+02:00
5,8796093030850,Monja,Tsiranana,549758486491,"About Rumi, glish-speaking world simply as Rum...",2010-10-12T10:34:23.553000000+02:00
6,2199023264850,Gheorghe,Popescu,549755938337,thanks,2010-10-12T06:56:32.295000000+02:00
7,4398046517420,Irina,Markov,549756749419,"About Jorge Luis Borges, universal de la inAbo...",2010-10-12T06:09:03.624000000+02:00
8,8796093030850,Monja,Tsiranana,549758482166,"About Rumi, al sphere of Khorasan, the eastern...",2010-10-11T19:52:56.133000000+02:00
9,8796093030850,Monja,Tsiranana,549758487306,"About Rumi, ther Pakistani languages written i...",2010-10-11T09:37:58.659000000+02:00


## Interactive / complex / 3

In [ ]:
q = """
// Q3. Friends and friends of friends that have been to given countries
/*
:params {
  personId: 6597069766734,
  countryXName: "Angola",
  countryYName: "Colombia",
  startDate: 1275393600000,
  endDate: 1277812800000
}
*/
MATCH (countryX:Country {name: $countryXName }),
      (countryY:Country {name: $countryYName }),
      (person:Person {id: $personId })
WITH person, countryX, countryY
LIMIT 1
MATCH (city:City)-[:IS_PART_OF]->(country:Country)
WHERE country IN [countryX, countryY]
WITH person, countryX, countryY, collect(city) AS cities
MATCH (person)-[:KNOWS*1..2]-(friend)-[:IS_LOCATED_IN]->(city)
WHERE NOT person=friend AND NOT city IN cities
WITH DISTINCT friend, countryX, countryY
MATCH (friend)<-[:HAS_CREATOR]-(message),
      (message)-[:IS_LOCATED_IN]->(country)
WHERE $endDate > message.creationDate >= $startDate AND
      country IN [countryX, countryY]
WITH friend,
     CASE WHEN country=countryX THEN 1 ELSE 0 END AS messageX,
     CASE WHEN country=countryY THEN 1 ELSE 0 END AS messageY
WITH friend, sum(messageX) AS xCount, sum(messageY) AS yCount
WHERE xCount>0 AND yCount>0
RETURN friend.id AS friendId,
       friend.firstName AS friendFirstName,
       friend.lastName AS friendLastName,
       xCount,
       yCount,
       xCount + yCount AS xyCount
ORDER BY xyCount DESC, friendId ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 4

In [ ]:
q = """
// Q4. New topics
/*
:params { personId: 4398046511333, startDate: 1275350400000, endDate: 1277856000000 }
*/
MATCH (person:Person {id: $personId })-[:KNOWS]-(friend:Person),
      (friend)<-[:HAS_CREATOR]-(post:Post)-[:HAS_TAG]->(tag)
WITH DISTINCT tag, post
WITH tag,
     CASE
       WHEN $endDate > post.creationDate >= $startDate THEN 1
       ELSE 0
     END AS valid,
     CASE
       WHEN $startDate > post.creationDate THEN 1
       ELSE 0
     END AS inValid
WITH tag, sum(valid) AS postCount, sum(inValid) AS inValidPostCount
WHERE postCount>0 AND inValidPostCount=0
RETURN tag.name AS tagName, postCount
ORDER BY postCount DESC, tagName ASC
LIMIT 10
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 5

In [ ]:
q = """
// Q5. New groups
/*
:params { personId: 6597069766734, minDate: 1288612800000 }
*/
MATCH (person:Person { id: $personId })-[:KNOWS*1..2]-(otherPerson)
WHERE
    person <> otherPerson
WITH DISTINCT otherPerson
MATCH (otherPerson)<-[membership:HAS_MEMBER]-(forum)
WHERE
    membership.creationDate > $minDate
WITH
    forum,
    collect(otherPerson) AS otherPersons
OPTIONAL MATCH (otherPerson2)<-[:HAS_CREATOR]-(post)<-[:CONTAINER_OF]-(forum)
WHERE
    otherPerson2 IN otherPersons
WITH
    forum,
    count(post) AS postCount
RETURN
    forum.title AS forumName,
    postCount
ORDER BY
    postCount DESC,
    forum.id ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 6

In [ ]:
q = """
// Q6. Tag co-occurrence
/*
:params { personId: 4398046511333, tagName: "Carl_Gustaf_Emil_Mannerheim" }
*/
MATCH (knownTag:Tag { name: $tagName })
WITH knownTag.id as knownTagId

MATCH (person:Person { id: $personId })-[:KNOWS*1..2]-(friend)
WHERE NOT person=friend
WITH
    knownTagId,
    collect(distinct friend) as friends
UNWIND friends as f
    MATCH (f)<-[:HAS_CREATOR]-(post:Post),
          (post)-[:HAS_TAG]->(t:Tag{id: knownTagId}),
          (post)-[:HAS_TAG]->(tag:Tag)
    WHERE NOT t = tag
    WITH
        tag.name as tagName,
        count(post) as postCount
RETURN
    tagName,
    postCount
ORDER BY
    postCount DESC,
    tagName ASC
LIMIT 10
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 7

In [ ]:
q = """
// Q7. Recent likers
/*
:params { personId: 4398046511268 }
*/
MATCH (person:Person {id: $personId})<-[:HAS_CREATOR]-(message:Message)<-[like:LIKES]-(liker:Person)
    WITH liker, message, like.creationDate AS likeTime, person
    ORDER BY likeTime DESC, toInteger(message.id) ASC
    WITH liker, head(collect({msg: message, likeTime: likeTime})) AS latestLike, person
RETURN
    liker.id AS personId,
    liker.firstName AS personFirstName,
    liker.lastName AS personLastName,
    latestLike.likeTime AS likeCreationDate,
    latestLike.msg.id AS commentOrPostId,
    coalesce(latestLike.msg.content, latestLike.msg.imageFile) AS commentOrPostContent,
    toInteger(floor(toFloat(latestLike.likeTime - latestLike.msg.creationDate)/1000.0)/60.0) AS minutesLatency,
    not((liker)-[:KNOWS]-(person)) AS isNew
ORDER BY
    likeCreationDate DESC,
    toInteger(personId) ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 8

In [ ]:
q = """
// Q8. Recent replies
/*
:params { personId: 143 }
*/
MATCH (start:Person {id: $personId})<-[:HAS_CREATOR]-(:Message)<-[:REPLY_OF]-(comment:Comment)-[:HAS_CREATOR]->(person:Person)
RETURN
    person.id AS personId,
    person.firstName AS personFirstName,
    person.lastName AS personLastName,
    comment.creationDate AS commentCreationDate,
    comment.id AS commentId,
    comment.content AS commentContent
ORDER BY
    commentCreationDate DESC,
    commentId ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 9

In [ ]:
q = """
// Q9. Recent messages by friends or friends of friends
/*
:params { personId: 4398046511268, maxDate: 1289908800000 }
*/
MATCH (root:Person {id: $personId })-[:KNOWS*1..2]-(friend:Person)
WHERE NOT friend = root
WITH collect(distinct friend) as friends
UNWIND friends as friend
    MATCH (friend)<-[:HAS_CREATOR]-(message:Message)
    WHERE message.creationDate < $maxDate
RETURN
    friend.id AS personId,
    friend.firstName AS personFirstName,
    friend.lastName AS personLastName,
    message.id AS commentOrPostId,
    coalesce(message.content,message.imageFile) AS commentOrPostContent,
    message.creationDate AS commentOrPostCreationDate
ORDER BY
    commentOrPostCreationDate DESC,
    message.id ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 10

In [ ]:
q = """
// Q10. Friend recommendation
/*
:params { personId: 4398046511333, month: 5 }
*/
MATCH (person:Person {id: $personId})-[:KNOWS*2..2]-(friend),
       (friend)-[:IS_LOCATED_IN]->(city:City)
WHERE NOT friend=person AND
      NOT (friend)-[:KNOWS]-(person)
WITH person, city, friend, datetime({epochMillis: friend.birthday}) as birthday
WHERE  (birthday.month=$month AND birthday.day>=21) OR
        (birthday.month=($month%12)+1 AND birthday.day<22)
WITH DISTINCT friend, city, person
OPTIONAL MATCH (friend)<-[:HAS_CREATOR]-(post:Post)
WITH friend, city, collect(post) AS posts, person
WITH friend,
     city,
     size(posts) AS postCount,
     size([p IN posts WHERE (p)-[:HAS_TAG]->()<-[:HAS_INTEREST]-(person)]) AS commonPostCount
RETURN friend.id AS personId,
       friend.firstName AS personFirstName,
       friend.lastName AS personLastName,
       commonPostCount - (postCount - commonPostCount) AS commonInterestScore,
       friend.gender AS personGender,
       city.name AS personCityName
ORDER BY commonInterestScore DESC, personId ASC
LIMIT 10
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 11

In [ ]:
q = """
// Q11. Job referral
/*
:params {
  personId: 10995116277918,
  countryName: "Hungary",
  workFromYear: 2011
}
*/
MATCH (person:Person {id: $personId })-[:KNOWS*1..2]-(friend:Person)
WHERE not(person=friend)
WITH DISTINCT friend
MATCH (friend)-[workAt:WORK_AT]->(company:Company)-[:IS_LOCATED_IN]->(:Country {name: $countryName })
WHERE workAt.workFrom < $workFromYear
RETURN
        friend.id AS personId,
        friend.firstName AS personFirstName,
        friend.lastName AS personLastName,
        company.name AS organizationName,
        workAt.workFrom AS organizationWorkFromYear
ORDER BY
        organizationWorkFromYear ASC,
        toInteger(personId) ASC,
        organizationName DESC
LIMIT 10
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 12

In [ ]:
q = """
// Q12. Expert search
/*
:params { personId: 10995116278009, tagClassName: "Monarch" }
*/
MATCH (tag:Tag)-[:HAS_TYPE|IS_SUBCLASS_OF*0..]->(baseTagClass:TagClass)
WHERE tag.name = $tagClassName OR baseTagClass.name = $tagClassName
WITH collect(tag.id) as tags
MATCH (:Person {id: $personId })-[:KNOWS]-(friend:Person)<-[:HAS_CREATOR]-(comment:Comment)-[:REPLY_OF]->(:Post)-[:HAS_TAG]->(tag:Tag)
WHERE tag.id in tags
RETURN
    friend.id AS personId,
    friend.firstName AS personFirstName,
    friend.lastName AS personLastName,
    collect(DISTINCT tag.name) AS tagNames,
    count(DISTINCT comment) AS replyCount
ORDER BY
    replyCount DESC,
    toInteger(personId) ASC
LIMIT 20
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 13

In [ ]:
q = """
// Q13. Single shortest path
/*
:params { person1Id: 8796093022390, person2Id: 8796093022357 }
*/
MATCH
    (person1:Person {id: $person1Id}),
    (person2:Person {id: $person2Id}),
    path = shortestPath((person1)-[:KNOWS*]-(person2))
RETURN
    CASE path IS NULL
        WHEN true THEN -1
        ELSE length(path)
    END AS shortestPathLength
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## Interactive / complex / 14

In [ ]:
q = """
// Q14. Trusted connection paths
// Requires the Neo4j Graph Data Science library
/*
:params { person1Id: 14, person2Id: 27 }
*/
// Check whether a path exists -- if there is no path, the query will return an empty result
MATCH
    path = shortestPath((person1 {id: $person1Id})-[:KNOWS*]-(person2 {id: $person2Id}))

// ----------------------------------------------------------------------------------------------------
// the actual values are not important,
// we are only interested in whether there is a row or not
WITH 42 AS dummy
// ----------------------------------------------------------------------------------------------------

MATCH (person1:Person {id: $person1Id}), (person2:Person {id: $person2Id})
CALL gds.graph.project.cypher(
  apoc.create.uuidBase64(),
  'MATCH (p:Person) RETURN id(p) AS id',
  'MATCH
      (pA:Person)-[knows:KNOWS]-(pB:Person),
      (pA)<-[:HAS_CREATOR]-(m1:Message)-[r:REPLY_OF]-(m2:Message)-[:HAS_CREATOR]->(pB)
    WITH
      id(pA) AS source,
      id(pB) AS target,
      count(r) AS numInteractions
    RETURN
      source,
      target,
      CASE WHEN round(40-sqrt(numInteractions)) > 1 THEN round(40-sqrt(numInteractions)) ELSE 1 END AS weight
  '
)
YIELD graphName

// ----------------------------------------------------------------------------------------------------
WITH person1, person2, graphName
// ----------------------------------------------------------------------------------------------------

CALL gds.shortestPath.dijkstra.stream(
    graphName, {sourceNode: person1, targetNode: person2, relationshipWeightProperty: 'weight'}
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path

WITH path, totalCost, graphName

CALL gds.graph.drop(graphName, false)
YIELD graphName as graphNameremoved

RETURN [person IN nodes(path) | person.id] AS personIdsInPath, totalCost AS pathWeight
LIMIT 1
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 1

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 2

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 3

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 4

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 5

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 6

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 7

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 8

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 9

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 10

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 11

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 12

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 13

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 14

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 15

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 16

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 17

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 18

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 19

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI / Read / 20

In [ ]:
q = """
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, par1=111, par2=111)
        df = pd.DataFrame(result.data())
df

,shortestPathLength
0,3


## BI Read 17

In [3]:
q = """
MATCH
  (tag:Tag {name: $tag}),
  (person1:Person)<-[:HAS_CREATOR]-(message1:Message)-[:REPLY_OF*0..]->(post1:Post)<-[:CONTAINER_OF]-(forum1:Forum),
  (message1)-[:HAS_TAG]->(tag),
// Having two HAS_MEMBER edges in the same MATCH clause ensures that person2 and person3 are different
// as Cypher's edge-isomorphic matching does not allow for such a match in a single MATCH clause.
  (forum1)<-[:HAS_MEMBER]->(person2:Person)<-[:HAS_CREATOR]-(comment:Comment)-[:HAS_TAG]->(tag),
  (forum1)<-[:HAS_MEMBER]->(person3:Person)<-[:HAS_CREATOR]-(message2:Message),
  (comment)-[:REPLY_OF]->(message2)-[:REPLY_OF*0..]->(post2:Post)<-[:CONTAINER_OF]-(forum2:Forum)
// The query allows message2 = post2. If this is the case, their HAS_TAG edges to tag overlap,
// and Cypher's edge-isomorphic matching does not allow for such a match in a single MATCH clause.
// To work around this, we add them in separate MATCH clauses.
MATCH (comment)-[:HAS_TAG]->(tag)
MATCH (message2)-[:HAS_TAG]->(tag)
WHERE forum1 <> forum2
  AND message2.creationDate > message1.creationDate + duration({hours: $delta})
  AND NOT (forum2)-[:HAS_MEMBER]->(person1)
RETURN person1.id, count(DISTINCT message2) AS messageCount
ORDER BY messageCount DESC, person1.id ASC
LIMIT 10
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q, tag='Slavoj_Žižek', delta=4)
        df = pd.DataFrame(result.data())
df

,person1.id,messageCount
0,10349,18
1,2199023260496,14
2,2199023263915,14
3,2199023264767,14
4,6597069769521,14
5,10995116278639,14
6,8749,13
7,4398046512698,13
8,6597069767602,13
9,8796093026049,13
